In [1]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time
# </snippet_imports>

'''
Authenticate
Authenticates your credentials and creates a client.
'''
# <snippet_vars>
subscription_key = "d5d0e372874a4c11ba463a30abe853f1"
endpoint = "https://answer-sheet.cognitiveservices.azure.com/"
# </snippet_vars>
# </snippet_imports_and_vars>

# <snippet_client>
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

In [2]:
from PIL import Image, ImageDraw
read_image = open("../images/vishnu-white.png", "rb")
# image = Image.open(read_image)

# draw = ImageDraw.Draw(image)
# Call API with image and raw response (allows you to get the operation location)
read_response = computervision_client.read_in_stream(read_image, raw=True)
# Get the operation location (URL with ID as last appendage)
read_operation_location = read_response.headers["Operation-Location"]
# Take the ID off and use to get results
operation_id = read_operation_location.split("/")[-1]

# Call the "GET" API and wait for the retrieval of the results
while True:
    read_result = computervision_client.get_read_result(operation_id)
    if read_result.status.lower () not in ['notstarted', 'running']:
        break
    print ('Waiting for result...')
    time.sleep(1)

Waiting for result...


In [3]:
import cv2
import numpy as np
from PIL import Image, ImageFilter,ImageOps
image = Image.open(read_image)
draw = ImageDraw.Draw(image)


counter=0
lineSum = 0
topbox = 0
bottombox = 0
# Print results, line by line
if read_result.status == OperationStatusCodes.succeeded:
    for text_result in read_result.analyze_result.read_results:
        for line in text_result.lines:
            #print(line.text)
            lineSum += line.bounding_box[0]
            #print(line.bounding_box)
            if counter==0:topbox=line.bounding_box[1]
            bottombox=line.bounding_box[5]
            counter+=1

  
img = image.convert('RGB') 
# Read image
img = np.array(img) 
height, width, channels = img.shape
print(height,width)
# Convert image to grayscale
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# Use canny edge detection
edges = cv2.Canny(gray,50,150,apertureSize=3)

# Apply HoughLinesP method to
# to directly obtain line end points
lines = cv2.HoughLinesP(
			edges, # Input edge image
			1, # Distance resolution in pixels
			np.pi/180, # Angle resolution in radians
			threshold=100, # Min number of votes for valid line
			minLineLength=5, # Min allowed length of line
			maxLineGap=10 # Max allowed gap between line for joining them
			)
lines_list=[]

# Iterate over points
for points in lines:
	# Extracted points nested in the list
	x1,y1,x2,y2=points[0]
	# Draw the lines joing the points
	# On the original image
	cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)
	# Maintain a simples lookup list for points
	lines_list.append([(x1,y1),(x2,y2)])

cv2.imshow("hello",img)
cv2.waitKey(1)
marginLineLeft = 0
breakLoop = False
marginLineLeft = 25
for top in range(10,int(topbox)):
    if(breakLoop):
        break
    for left in range(30,int(lineSum)//counter+10):
        if(img[left][top][1] > 250):
            breakLoop = True
            marginLineLeft = left
            break
            
breakLoop = False
# for top in range(int(bottombox),image.height-10):
#     if(breakLoop):
#         break
#     for left in range(10,int(lineSum)//counter+10):
#         if(img[top][left][1] > 250):
#             breakLoop = True
#             marginLineLeft = (left+marginLineLeft)//2
#             break
print(lines_list[0],lines_list[1])
print(marginLineLeft)
draw.line((marginLineLeft,0,marginLineLeft,height-1), fill=(255,0,0))
for text_result in read_result.analyze_result.read_results:
    for line in text_result.lines:
        
        coordinates = ((line.bounding_box[0], line.bounding_box[1]),
                           (line.bounding_box[4], line.bounding_box[5]))
        draw.rectangle(coordinates, outline=(0,150,0))
        if line.bounding_box[0]<marginLineLeft:
            coordinates = ((line.bounding_box[0], line.bounding_box[1]),
                           (line.bounding_box[4], line.bounding_box[5]))
            draw.rectangle(coordinates, outline=(255,0,0))
image.show()
print()

1270 904
[(102, 276), (104, 447)] [(902, 416), (902, 126)]
25


TypeError: color must be int or single-element tuple